Library to be used with pandas

In [ ]:
#required so that pandas can work with excel file easily
!pip install xlsxwriter

Group Based on the Sequence (Temp.xlsx)

In [2]:
import pandas as pd

df=pd.read_csv("wt8c.csv")
df = df[['Start\r\nAA', 'End\r\nAA', 'Var. Pos.\r\nProtein', 'Sequence',
       'Glycans', 'Validate', 'Score', 'XIC area\r\nsummed']]

df['Sequence']=df['Sequence'].str.upper()                                                               # making sequence words into upper case so grouping is easier - Unoccupied glycans do not have lower case N so this allows occ and unocc glycans to be grouped together

df2=pd.DataFrame()
uni_col=df['Sequence'].unique()                                                                         # finding the unique sequences and save as variable uni_col
columns=df.columns                                                                                      # getting the column name for the order when exporting file
writer = pd.ExcelWriter("Temp.xlsx", engine = 'xlsxwriter') #making file to write in

#loop over unquie column names,find in sheet and write into new file
for col in uni_col:
    found_df=df[df['Sequence'] == col] #matching based on the column name to get rows
    found_df[columns].to_excel(writer,sheet_name=col,index=False)
writer.save()
writer.close()
print("Completed")

Completed


In [ ]:
columns

#print(range(1, len(df['Sequence'].unique())))

Calculating the percentage (Temp-2.xlsx)

In [ ]:
import pandas as pd
raw_df=pd.ExcelFile("Temp.xlsx")
sheet_names=raw_df.sheet_names #getting sheetnames so i go over each sheet
writer = pd.ExcelWriter("Temp-2.xlsx", engine = 'xlsxwriter')

for sheet_name in sheet_names:
    print(sheet_name)
    df=pd.read_excel("Temp.xlsx",sheet_name=sheet_name)
    total=df['XIC area_x000D_\nsummed'].sum() #getting the total of the XIC area

    #going over each value of XIC and finding the percentage
    for i,rows in df.iterrows():
        df.at[i,'Percentage']=round((rows['XIC area_x000D_\nsummed']/total)*100,2)
    df.to_excel(writer,sheet_name=sheet_name,index=False)

writer.save()
writer.close()
print("Completed")

Finding the classication (Temp-3.xlsx)

In [ ]:
import pandas as pd
raw_df=pd.ExcelFile("Temp-2.xlsx")
classification_df=pd.read_csv("Glycan_s.csv")
sheet_names=raw_df.sheet_names
writer = pd.ExcelWriter("Temp-3.xlsx", engine = 'xlsxwriter')

for sheet_name in sheet_names:
    print(sheet_name)
    df=pd.read_excel("Temp-2.xlsx",sheet_name=sheet_name)

    #for each row i try to find the matched Gylcan in the Glycan_s file
    for i,rows in df.iterrows():
        print(rows['Glycans'])
        #here is where the main work starts to find the right classfication
        class_name=classification_df[classification_df['Glycans'] == rows['Glycans']]
        print(class_name['Species'])
        #then its just a matter of assigning the values
        try:
          df.at[i,'Classification']=class_name.iloc[0]['Species']
        except:
          df.at[i,'Classification']='Unoccupied'
          # continue
    df.to_excel(writer,sheet_name=sheet_name,index=False)
writer.save()
writer.close()
print("Completed")

Calculating Values like M8 (Temp-4.xlsx)

In [ ]:
import pandas as pd
raw_df=pd.ExcelFile("Temp-3.xlsx")
#i got the names of the values i need to calculate
to_calc=['M9Glc','M9','M8','M7','M6','M5','M4','M3','FM','HYBRID','FHYBRID','HexNAc(3)(x)','HexNAc(3)(F)(x)','HexNAc(4)(x)','HexNAc(4)(F)(x)'
,'HexNAc(5)(x)','HexNAc(5)(F)(x)','HexNAc(6+)(x)','HexNAc(6+)(F)(x)','Unoccupied','Core']
sheet_names=raw_df.sheet_names
writer = pd.ExcelWriter("Temp-4.xlsx", engine = 'xlsxwriter')

for sheet_name in sheet_names:
    print(sheet_name)
    df=pd.read_excel("Temp-3.xlsx",sheet_name=sheet_name)

    for index,calc in enumerate(to_calc):
        #i try to find if those names above(like M8) appear in the table and in which rows
        matched_records=df[df['Classification'] == calc]
        #if i do find 1 or more rows with it then i add them and assign it else its 0
        if(not matched_records.empty):
          value=matched_records['Percentage'].sum()
          df.at[index,'Random Name']=calc
          df.at[index,'Random value']=value
        else:
          df.at[index,'Random Name']=calc
          df.at[index,'Random value']=0.00
    df.to_excel(writer,sheet_name=sheet_name,index=False)
    
writer.save()
writer.close()
print("Completed")

Calculating Stuff like Fucose (Temp-5.xlsx)

In [ ]:
import pandas as pd
raw_df=pd.ExcelFile("Temp-4.xlsx")
#name of the things to calculate
to_calc=['Oligomannose','Hybrid','Complex','Unoccupied','Core','Fucose','NeuAc/NeuGc']
sheet_names=raw_df.sheet_names
writer = pd.ExcelWriter("Temp-5.xlsx", engine = 'xlsxwriter')

for sheet_name in sheet_names:
    print(sheet_name)
    df=pd.read_excel("Temp-4.xlsx",sheet_name=sheet_name)
    #from oligomannose to core the calculation is done based on the row index
    #as i know from which row i need to get the values
    oli=df.iloc[1:9]['Random value'].sum()
    df.at[0,'Oligomannose']=oli
    hybrid=df.iloc[9:11]['Random value'].sum()
    #print(hybrid)
    df.at[0,'Hybrid']=hybrid
    comple=df.iloc[11:19]['Random value'].sum()
    df.at[0,'Complex']=comple
    unocc=df.iloc[19]['Random value']
    df.at[0,'Unoccupied']=unocc
    core=df.iloc[20]['Random value']
    df.at[0,'Core']=core

    #for Fucose i just try to find "F" in the string and if it exist i add it to total
    total=0
    for i,rows in df.iterrows():
        if(str(rows['Random Name']).find("F") != -1):
          total+=rows['Random value']
    df.at[0,'Fucose']=total

    #for Neu i do the same but with those keywords and an OR operator
    total=0
    for i,rows in df.iterrows():
        if((str(rows['Glycans']).find("NeuAc") != -1) or (str(rows['Glycans']).find("NeuGc") != -1)):
          print(i)
          total+=rows['Percentage']
    df.at[0,'NeuAc/NeuGc']=total
    df.to_excel(writer,sheet_name=sheet_name,index=False)
    
writer.save()
writer.close()
print("Completed")

Rename Sheet names (raw_data_analysed.xlsm)

In [ ]:
import pandas as pd
df=pd.ExcelFile("Temp-5.xlsx")
sheet_names=df.sheet_names
new_col_name_df=pd.read_csv("SOSIP_pngs.csv") # got the data read from the new name file
writer = pd.ExcelWriter("raw_data_analysed.xlsx", engine = 'xlsxwriter')

for sheet_name in sheet_names:
    print(sheet_name)
    df=pd.read_excel("Temp-5.xlsx",sheet_name=sheet_name)
    #i checked sheet to see which Protein it is
    pos=df.iloc[0]['Var. Pos._x000D_\nProtein']
    index=1
    while(str(pos) == 'nan'): #this loop is only here if there is unocc at start hence it needs to go down to find valid Pos
          pos=df.iloc[index]['Var. Pos._x000D_\nProtein']
          if(str(pos) != 'nan'): #basically "nan" in pandas means blank cell
            break

    #i check the pos against the new name file and then assgin it
    found_row=new_col_name_df[new_col_name_df['Pos'] == pos]
    if(found_row.iloc[0]['Hxb2 Env coords'].find("N") == -1):
      #if the new sheet name has N then i wont add N else i will
      sheet_name="N"+found_row.iloc[0]['Hxb2 Env coords']
    else:
      sheet_name=found_row.iloc[0]['Hxb2 Env coords']
    df.to_excel(writer,sheet_name=sheet_name,index=False)

writer.save()
writer.close()
print("COmpleted")